In [21]:
!pip install doclayout-yolo

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [16]:
from pdf2image import convert_from_path
from PIL import Image
import time
start = time.time()
# Конвертация PDF в изображения
images = convert_from_path("/home/jovyan/danila_workflow/nn_cso_2023_rus.pdf")
for ide, i in enumerate(images):
    i.save(f"/home/jovyan/danila_workflow/nn_cso_2023_rus/page_{ide}.jpg", "JPEG")
print(time.time() - start)

119.75924491882324


In [22]:
import cv2
from doclayout_yolo import YOLOv10

# Load the pre-trained model
model = YOLOv10("/home/jovyan/danila_workflow/model/doclayout_yolo_docstructbench_imgsz1024.pt")

/home/jovyan/.local/lib/python3.11/site-packages/doclayout_yolo/nn/tasks.py:733: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(file, map_location="cpu")


In [23]:
for i in range(len(images)):
    path = f"/home/jovyan/danila_workflow/nn_cso_2023_rus/page_{i}.jpg"
    # Perform prediction
    det_res = model.predict(
        path,   # Image to predict
        # imgsz=1024,        # Prediction image size
        conf=0.2,          # Confidence threshold
        device="cuda:0"    # Device to use (e.g., 'cuda:0' or 'cpu')
    )
    
    # Annotate and save the result
    annotated_frame = det_res[0].plot(pil=True, line_width=5, font_size=20)
    cv2.imwrite(f"/home/jovyan/danila_workflow/done_nn_cso_2023_rus/page_{i}.jpg", annotated_frame)


image 1/1 /home/jovyan/danila_workflow/nn_cso_2023_rus/page_0.jpg: 1024x832 1 title, 3 abandons, 1 figure, 17.1ms
Speed: 3.3ms preprocess, 17.1ms inference, 0.7ms postprocess per image at shape (1, 3, 1024, 832)

image 1/1 /home/jovyan/danila_workflow/nn_cso_2023_rus/page_1.jpg: 672x1024 18 titles, 59 plain texts, 1 abandon, 1 figure, 19.4ms
Speed: 3.5ms preprocess, 19.4ms inference, 0.7ms postprocess per image at shape (1, 3, 672, 1024)

image 1/1 /home/jovyan/danila_workflow/nn_cso_2023_rus/page_2.jpg: 672x1024 8 titles, 21 plain texts, 3 abandons, 13 figures, 17.7ms
Speed: 3.3ms preprocess, 17.7ms inference, 0.7ms postprocess per image at shape (1, 3, 672, 1024)

image 1/1 /home/jovyan/danila_workflow/nn_cso_2023_rus/page_3.jpg: 672x1024 2 titles, 2 plain texts, 3 abandons, 1 figure, 17.0ms
Speed: 2.9ms preprocess, 17.0ms inference, 0.6ms postprocess per image at shape (1, 3, 672, 1024)

image 1/1 /home/jovyan/danila_workflow/nn_cso_2023_rus/page_4.jpg: 672x1024 3 titles, 19 plain 

In [24]:
import cv2
from doclayout_yolo import YOLOv10
from sklearn.cluster import KMeans
import numpy as np
import os

# Параметры
NUM_CLUSTERS = 4
CONFIDENCE_THRESHOLD = 0.2
DEVICE = "cuda:0"  # Или "cpu"

# Цвета для кластеров (можно добавить больше, если нужно)
CLUSTER_COLORS = [
    (255, 0, 0),    # Синий
    (0, 255, 0),    # Зеленый
    (0, 0, 255),    # Красный
    (255, 255, 0),  # Желтый
    # Добавьте больше цветов при необходимости
]

# Пути к директориям
MODEL_PATH = "/home/jovyan/danila_workflow/model/doclayout_yolo_docstructbench_imgsz1024.pt"
INPUT_DIR = "/home/jovyan/danila_workflow/nn_cso_2023_rus/"
OUTPUT_DIR = "/home/jovyan/danila_workflow/test_done_nn_cso_2023_rus/"

# Создаем выходную директорию, если ее нет
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Загружаем модель
model = YOLOv10(MODEL_PATH)

# Получаем список изображений
images = sorted([f for f in os.listdir(INPUT_DIR) if f.lower().endswith('.jpg')])

/home/jovyan/.local/lib/python3.11/site-packages/doclayout_yolo/nn/tasks.py:733: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(file, map_location="cpu")


In [25]:
for i, image_name in enumerate(images):
    image_path = os.path.join(INPUT_DIR, image_name)
    
    # Выполняем предсказание
    det_res = model.predict(
        image_path,
        conf=CONFIDENCE_THRESHOLD,
        device=DEVICE
    )
    
    # Проверяем, есть ли детекции
    if len(det_res) == 0 or len(det_res[0].boxes) == 0:
        print(f"Нет детекций для изображения {image_name}")
        continue
    
    # Извлекаем bounding box'ы
    boxes = det_res[0].boxes.xyxy  # Предполагается, что формат [x1, y1, x2, y2]
    
    # Конвертируем в numpy массив, если необходимо
    if isinstance(boxes, list):
        boxes = np.array(boxes)
    elif not isinstance(boxes, np.ndarray):
        boxes = boxes.cpu().numpy()
    
    # Вычисляем центры bounding box'ов
    centers = []
    for box in boxes:
        x1, y1, x2, y2 = box[:4]
        center_x = (x1 + x2) / 2
        center_y = (y1 + y2) / 2
        centers.append([center_x, center_y])
    
    centers = np.array(centers)
    
    # Проверяем количество детекций
    if len(centers) < 1:
        print(f"Нет центров для кластеризации в изображении {image_name}")
        continue
    
    # Определяем количество кластеров (не больше количества детекций)
    current_clusters = min(NUM_CLUSTERS, len(centers))
    
    # Выполняем кластеризацию KMeans
    kmeans = KMeans(n_clusters=current_clusters, random_state=42)
    kmeans.fit(centers)
    labels = kmeans.labels_
    
    # Присваиваем цвета кластерам
    assigned_colors = [CLUSTER_COLORS[label % len(CLUSTER_COLORS)] for label in labels]
    
    # Загружаем изображение с помощью OpenCV
    image = cv2.imread(image_path)
    if image is None:
        print(f"Не удалось загрузить изображение {image_path}")
        continue
    
    # Рисуем bounding box'ы с цветами кластеров
    for box, color in zip(boxes, assigned_colors):
        x1, y1, x2, y2 = map(int, box[:4])  # Преобразуем координаты в целые числа
        cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
    
    # Сохраняем аннотированное изображение
    output_path = os.path.join(OUTPUT_DIR, image_name)
    cv2.imwrite(output_path, image)
    
    print(f"Обработано изображение {image_name}")


image 1/1 /home/jovyan/danila_workflow/nn_cso_2023_rus/page_0.jpg: 1024x832 1 title, 3 abandons, 1 figure, 21.3ms
Speed: 3.3ms preprocess, 21.3ms inference, 0.6ms postprocess per image at shape (1, 3, 1024, 832)
Обработано изображение page_0.jpg

image 1/1 /home/jovyan/danila_workflow/nn_cso_2023_rus/page_1.jpg: 672x1024 18 titles, 59 plain texts, 1 abandon, 1 figure, 17.4ms
Speed: 2.8ms preprocess, 17.4ms inference, 0.6ms postprocess per image at shape (1, 3, 672, 1024)
Обработано изображение page_1.jpg

image 1/1 /home/jovyan/danila_workflow/nn_cso_2023_rus/page_10.jpg: 672x1024 5 titles, 7 plain texts, 5 abandons, 4 figures, 1 figure_caption, 16.9ms
Speed: 2.7ms preprocess, 16.9ms inference, 0.7ms postprocess per image at shape (1, 3, 672, 1024)
Обработано изображение page_10.jpg

image 1/1 /home/jovyan/danila_workflow/nn_cso_2023_rus/page_100.jpg: 672x1024 11 titles, 12 plain texts, 5 abandons, 1 figure, 17.0ms
Speed: 3.1ms preprocess, 17.0ms inference, 0.6ms postprocess per image

In [20]:
print("Кластеризация и аннотирование завершены.")

Кластеризация и аннотирование завершены.
